In [1]:
import pandas as pd
import numpy as np
from prophet import Prophet
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
import warnings
import logging
import matplotlib.pyplot as plt 
import os 
logging.getLogger('cmdstanpy').setLevel(logging.ERROR)
warnings.filterwarnings("ignore")

c:\Users\Tuan Ank\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
# --- 1. CẤU HÌNH CÁC NHÓM (GROUPS) ---
family_groups = {
    "Ultra Low": [
        "HOME APPLIANCES", "BABY CARE", "BOOKS"
    ],
    "Low": [
        "LINGERIE", "CELEBRATION", "PLAYERS AND ELECTRONICS", "AUTOMOTIVE",
        "LADIESWEAR", "PET SUPPLIES", "LAWN AND GARDEN", "BEAUTY",
        "SCHOOL AND OFFICE SUPPLIES", "MAGAZINES", "HARDWARE",
    ],
    "Medium": [
        "PREPARED FOODS", "LIQUOR,WINE,BEER","HOME AND KITCHEN I", 
        "GROCERY II", "SEAFOOD", "HOME AND KITCHEN II"
    ],
    "High": [
        "BREAD/BAKERY", "POULTRY", "PERSONAL CARE", "MEATS",
        "DELI", "EGGS", "HOME CARE", "FROZEN FOODS", "GROCERY I",
        "BEVERAGES" , "PRODUCE", "CLEANING", "DAIRY"
    ]
}

n_test_days = 90 # Số ngày dùng để kiểm tra mô hình
filename = 'final_dataset.csv'
confidence_level = 0.95  # Độ tin cậy (0.95 tương ứng với 95%)

# --- Tạo thư mục lưu các biểu đồ ---
folder_components = 'Bieu_do_thanh_phan' # Thư mục cho biểu đồ thành phần
folder_forecast = 'Bieu_do_du_bao' # Thư mục cho biểu đồ so sánh
os.makedirs(folder_components, exist_ok=True)
os.makedirs(folder_forecast, exist_ok=True)

print(f"Đã cấu hình thư mục ảnh:\n - {folder_components}/\n - {folder_forecast}/")

Đã cấu hình thư mục ảnh:
 - Bieu_do_thanh_phan/
 - Bieu_do_du_bao/


In [3]:
# --- 2. HÀM TÍNH ACCURACY THEO BIÊN ĐỘ (TOLERANCE) ---
def calculate_tolerance_accuracy(y_true, y_pred, tolerance=0.2):
    """
    Tính % số ngày mà sai số nằm trong khoảng cho phép (mặc định 20%)
    Logic: |y_true - y_pred| <= tolerance * y_true
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    # Tính ngưỡng cho phép từng ngày
    # Dùng abs(y_true) để an toàn cho trường hợp sales bị âm (dù hiếm)
    thresholds = tolerance * np.abs(y_true)
    
    # Kiểm tra điều kiện: Sai số tuyệt đối <= Ngưỡng
    # Kết quả là mảng True/False
    is_accurate = np.abs(y_true - y_pred) <= thresholds
    
    # Tính trung bình (True=1, False=0) -> ra tỷ lệ phần trăm
    return np.mean(is_accurate)

In [6]:
# --- 3. NẠP DỮ LIỆU ---
try:
    df_master = pd.read_csv(filename)                               
    df_master['date'] = pd.to_datetime(df_master['date'])              
    print(f"Đã nạp dữ liệu thành công: {len(df_master)} dòng.")     
except FileNotFoundError:                                             
    raise SystemExit(f"Lỗi: Không tìm thấy file '{filename}'")      

Đã nạp dữ liệu thành công: 444576 dòng.


In [ ]:
# --- 4. VÒNG LẶP CHÍNH ---
print(f"\nBẮT ĐẦU CHẠY PROPHET (Interval Width: {confidence_level*100}%)...")
print("="*60)

all_families_results = [] 
all_group_averages = []   

for group_name, families in family_groups.items():
    print(f"\n>>> Đang xử lý nhóm: {group_name}")
    
    current_group_results = [] 
    
    for family in families:
        # --- 4a. Chuẩn bị dữ liệu ---
        df_family_raw = df_master[df_master['family'] == family]
        
        if df_family_raw.empty:
            print(f"  [Bỏ qua] Không tìm thấy dữ liệu cho '{family}'")
            continue
        mean_sales = np.mean(df_family_raw['sales'])
        # Tổng hợp theo ngày
        df_family = df_family_raw.groupby('date').agg( 
            sales=('sales', 'sum'),
            onpromotion=('onpromotion', 'sum'),
            is_holiday=('is_holiday', 'max'),
            dcoilwtico=('dcoilwtico', 'mean')
        ).reset_index()
        
        # Định dạng cho Prophet
        
        df_prophet = df_family.rename(columns={'date': 'ds', 'sales': 'y'})
        
        # Đảm bảo tần suất ngày & Fillna
        full_range = pd.date_range(start=df_prophet['ds'].min(), end=df_prophet['ds'].max(), freq='D') 
        df_prophet = pd.DataFrame({'ds': full_range}).merge(df_prophet, on='ds', how='left') 
        
        df_prophet['y'] = df_prophet['y'].fillna(0) 
        df_prophet['onpromotion'] = df_prophet['onpromotion'].fillna(0) 
        df_prophet['is_holiday'] = df_prophet['is_holiday'].fillna(0)
        df_prophet['dcoilwtico'] = df_prophet['dcoilwtico'].interpolate(limit_direction='both').fillna(0) 
        
        # Chia Train/Test
        if len(df_prophet) <= n_test_days: 
            print(f"  [Bỏ qua] Dữ liệu quá ngắn: {family}")
            continue
            
        train_prophet = df_prophet.iloc[:-n_test_days]  
        test_prophet = df_prophet.iloc[-n_test_days:] 
        
        # --- 4b. Huấn luyện Prophet ---
        try:
            
            model = Prophet(
                            weekly_seasonality=True, 
                            yearly_seasonality=True, 
                            seasonality_mode='additive', 
                            changepoint_prior_scale=0.05, # Thay đổi độ nhạy với đột biến 
                            seasonality_prior_scale=10.0, # Điều chỉnh mức độ phức tạp của mùa vụ
                            interval_width=confidence_level 
                            )
            # Thêm các biến ngoại sinh
            model.add_regressor('onpromotion') 
            model.add_regressor('is_holiday')
            model.add_regressor('dcoilwtico')

            model.fit(train_prophet) 
            
            future_df = test_prophet.drop(columns=['y'])
            forecast = model.predict(future_df)
            
            # Tạo tên file an toàn
            safe_family_name = family.replace("/", "_").replace(" ", "_")

            # --- A. VẼ VÀ LƯU COMPONENT PLOT ---
            fig_comp = model.plot_components(forecast) 
            plot_comp_filename = f"{folder_components}/{group_name}_{safe_family_name}_comp.png"
            plt.savefig(plot_comp_filename)
            plt.close(fig_comp) 
            
            # --- B. VẼ BIỂU ĐỒ ACTUAL vs FORECAST ---
            plt.figure(figsize=(12, 6))
            plt.plot(test_prophet['ds'], test_prophet['y'], label='Thực tế (Actual)', color='black', linewidth=2)
            plt.plot(forecast['ds'], forecast['yhat'], label='Dự báo (Forecast)', color='blue', linewidth=2,linestyle='--')
            
            # Tô màu vùng tin cậy (Confidence Interval)
            plt.fill_between(forecast['ds'], 
                            forecast['yhat_lower'], 
                            forecast['yhat_upper'], 
                            color='blue', alpha=0.2, 
                            label=f'Khoảng tin cậy {int(confidence_level*100)}%')
            
            plt.title(f"Dự báo vs Thực tế: {family} (Nhóm: {group_name})")
            plt.xlabel("Ngày")
            plt.ylabel("Doanh số (Sales)")
            plt.legend()
            plt.grid(True, alpha=0.3)
            
            # Lưu biểu đồ so sánh
            plot_forecast_filename = f"{folder_forecast}/{group_name}_{safe_family_name}_forecast.png"
            plt.savefig(plot_forecast_filename)
            plt.close() # Đóng figure
            
            # --- C. TÍNH TOÁN METRICS ---
            y_true = test_prophet['y'].values 
            y_pred = forecast['yhat'].values
            y_lower = forecast['yhat_lower'].values
            y_upper = forecast['yhat_upper'].values
            
            rmse = np.sqrt(mean_squared_error(y_true, y_pred))
            mae = mean_absolute_error(y_true, y_pred)
            mape = mean_absolute_percentage_error(y_true, y_pred)
            r2 = r2_score(y_true, y_pred)
            acc = calculate_tolerance_accuracy(y_true, y_pred, tolerance=0.20) * 100
            is_covered = (y_true >= y_lower) & (y_true <= y_upper)
            coverage = np.mean(is_covered) * 100
            avg_width = np.mean(y_upper - y_lower)
            mean_true = np.mean(y_true)
            if mean_true != 0:
                wape = (mae / mean_true)
            else:
                wape = 0.0
            print(f"  - {family}: R2={r2:.2f}, Acc={acc:.2f}%, Width={avg_width:.2f} | Đã lưu 2 plots")
            
            result_row = {
                'segment': group_name,  
                'family_name': family,
                'RMSE': rmse,                     # Sai số bình phương gốc trung bình
                'MAE': mae,                       # Độ lệch tuyệt đối trung bình
                'MAPE': mape,                     # Độ lệch phần trăm tuyệt đối trung bình
                'R2': r2,                         # Hệ số xác định
                'accuracy': acc, 
                'coverage_95': coverage,          # Tỷ lệ phần trăm điểm thực tế nằm trong khoảng tin cậy
                'avg_width': avg_width, # Chiều rộng trung bình của khoảng tin cậy  
                'mean_sales': mean_sales,           # Giá trị thực trung bình
                'WAPE': wape                      # Độ lệch phần trăm tuyệt đối
            }
            
            current_group_results.append(result_row) 
            all_families_results.append(result_row) 
            
        except Exception as e:
            print(f"  LỖI Prophet với {family}: {str(e)[:100]}...")

    # --- 4c. Tính Trung bình Nhóm ---
    if current_group_results: 
        df_curr_group = pd.DataFrame(current_group_results) 
        avg_metrics = df_curr_group.mean(numeric_only=True)
        
        avg_row = avg_metrics.to_dict() 
        avg_row['segment'] = group_name 
        avg_row['family_name'] = f"AVG_{group_name}" 
        
        all_group_averages.append(avg_row)
    else:
        print(f"  Không có kết quả nào cho nhóm {group_name}")



BẮT ĐẦU CHẠY PROPHET (Interval Width: 95.0%)...

>>> Đang xử lý nhóm: Ultra Low


09:55:06 - cmdstanpy - INFO - Chain [1] start processing
09:55:06 - cmdstanpy - INFO - Chain [1] done processing


  - HOME APPLIANCES: R2=0.13, Acc=18.89%, Width=10.98 | Đã lưu 2 plots


09:55:08 - cmdstanpy - INFO - Chain [1] start processing
09:55:08 - cmdstanpy - INFO - Chain [1] done processing


  - BABY CARE: R2=-1.62, Acc=8.89%, Width=7.64 | Đã lưu 2 plots


...

  - PRODUCE: R2=-0.48, Acc=67.78%, Width=22573.46 | Đã lưu 2 plots


09:56:09 - cmdstanpy - INFO - Chain [1] start processing
09:56:09 - cmdstanpy - INFO - Chain [1] done processing


  - CLEANING: R2=0.32, Acc=82.22%, Width=5974.98 | Đã lưu 2 plots


09:56:11 - cmdstanpy - INFO - Chain [1] start processing
09:56:11 - cmdstanpy - INFO - Chain [1] done processing


  - DAIRY: R2=0.76, Acc=97.78%, Width=3546.25 | Đã lưu 2 plots


In [50]:
# --- 5. TỔNG HỢP FINAL REPORT ---
print("\n" + "="*60)
print("ĐANG TẠO FILE BÁO CÁO TỔNG HỢP (MASTER REPORT)...")

if all_families_results:
    df_details = pd.DataFrame(all_families_results)
    df_groups_avg = pd.DataFrame(all_group_averages)
    
    grand_avg_metrics = df_details.mean(numeric_only=True)
    grand_avg_row = grand_avg_metrics.to_dict()
    grand_avg_row['segment'] = 'ALL_GROUPS'
    grand_avg_row['Family'] = 'GRAND_TOTAL_AVERAGE'
    
    df_grand_avg = pd.DataFrame([grand_avg_row])
    
    final_df = pd.concat([df_details, df_groups_avg, df_grand_avg], ignore_index=True)
    
    # Sắp xếp lại cột cho đẹp (thêm cột Avg_Confidence_Width)
    cols_order = [ 'family_name', 'MAE', 'RMSE', 'MAPE', 'R2', 'accuracy','coverage_95', 'avg_width','mean_sales','WAPE','segment']
    final_df = final_df[cols_order]
    
    final_filename = "PROPHET_result.csv"
    final_df.to_csv(final_filename, index=False, float_format='%.4f')
    
    print(f"Đã xuất file thành công: {final_filename}")
    print(f"Biểu đồ components: {folder_components}/")
    print(f"Biểu đồ so sánh:    {folder_forecast}/")
    print("-" * 40)
    print(final_df.tail(10).to_string(index=False)) 
else:
    print("Không có dữ liệu để xuất báo cáo.")


ĐANG TẠO FILE BÁO CÁO TỔNG HỢP (MASTER REPORT)...
Đã xuất file thành công: PROPHET_result.csv
Biểu đồ components: Bieu_do_thanh_phan/
Biểu đồ so sánh:    Bieu_do_du_bao/
----------------------------------------
  family_name         MAE         RMSE         MAPE        R2  accuracy  coverage_95    avg_width  mean_sales     WAPE    segment
    GROCERY I 4023.505707  6420.210378 1.208598e+18  0.414107 90.571429    94.285714 21284.615791 4209.822257 0.105293       High
    BEVERAGES 5401.824070  6958.830725 8.227084e+17  0.025262 60.571429    86.285714 19258.851581 2562.925178 0.186960       High
      PRODUCE 9586.289938 10485.231981 7.517148e+17 -6.448486  9.142857    71.428571 23676.558527 1455.648132 0.502819       High
     CLEANING 1742.503459  2523.646758 3.089068e+17 -0.037411 74.000000    79.428571  5612.202620 1244.611416 0.162495       High
        DAIRY  876.602232  1251.035394 2.529866e+17  0.360701 81.714286    92.000000  3342.404237  760.278800 0.119661       High
AVG_Ultr